In [1]:
import pandas as pd
PATH_to_file = './data/'
RANDOM_SEED = 42

### 1. Парсинг выполнен в отдельном ноутбуке. Данные от 2.12.2020

В этом разделе выполним предобработку тренировочного набора данных и объединение его с тестовым, для EDA и Feature Engineering 

In [2]:
# тренировочная выборка, полученная парсингом
df = pd.read_csv(PATH_to_file + 'all_auto_ru_02_12_2020.csv')

Посмотрим наименование и количество столбцов

In [3]:
print(df.columns)
len(df.columns)

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')


26

Загрузим тестовую выборку, чтоб сверить количество и наименование признаков

In [4]:
test = pd.read_csv(PATH_to_file + 'test.csv')

Проверим общие колонки и, возможно, с разным наименованием, но одним содержанием

In [5]:
df_col = df.columns
test_col = test.columns

In [6]:
common_col = []
t_col = []
count = 0
t = 0
for col in test_col:
    if col in df_col:
        common_col.append(col)
        count +=1
    else:
        t_col.append(col)
        t +=1
print('Количество общих столбцов и их наименование = ',count, common_col)
print('            ')
print('Количество столбцов содержащихся только в выборке Test и наименованиев = ', t, t_col)

Количество общих столбцов и их наименование =  21 ['bodyType', 'brand', 'color', 'description', 'engineDisplacement', 'enginePower', 'fuelType', 'mileage', 'modelDate', 'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']
            
Количество столбцов содержащихся только в выборке Test и наименованиев =  11 ['car_url', 'complectation_dict', 'equipment_dict', 'image', 'model_info', 'model_name', 'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor']


Посмотрим, какие столбцы не попали в категорию общих.

In [7]:
df_compare = df.drop(common_col, axis = 1)

In [8]:
df_compare.head()

,Комплектация,price,start_date,hidden,model
0,{'id': '0'},200000.0,2020-11-08T17:23:26Z,NaN,100
1,{'id': '0'},125000.0,2020-10-16T16:28:14Z,NaN,100
2,{'id': '0'},185000.0,2020-11-28T09:37:37Z,NaN,100
3,{'id': '0'},100000.0,2020-09-13T12:59:14Z,NaN,100
4,{'id': '0'},65000.0,2020-11-18T09:00:59Z,NaN,100


In [9]:
test_compare = test.drop(common_col, axis = 1)

In [10]:
test_compare.head()

,car_url,complectation_dict,equipment_dict,image,model_info,model_name,parsing_unixtime,priceCurrency,sell_id,super_gen,vendor
0,https://auto.ru/cars/used/sale/skoda/octavia/1...,NaN,"{""engine-proof"":true,""tinted-glass"":true,""airb...",https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1603226273,RUB,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",EUROPEAN
1,https://auto.ru/cars/used/sale/skoda/octavia/1...,NaN,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1603226277,RUB,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",EUROPEAN
2,https://auto.ru/cars/used/sale/skoda/superb/11...,"{""id"":""20026336"",""name"":""Ambition"",""available_...","{""cruise-control"":true,""tinted-glass"":true,""es...",https://avatars.mds.yandex.net/get-autoru-vos/...,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,1603226280,RUB,1100658222,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",EUROPEAN
3,https://auto.ru/cars/used/sale/skoda/octavia/1...,"{""id"":""20803582"",""name"":""Ambition"",""available_...","{""cruise-control"":true,""roller-blind-for-rear-...",https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1603226284,RUB,1100937408,"{""id"":""20105521"",""displacement"":1598,""engine_t...",EUROPEAN
4,https://auto.ru/cars/used/sale/skoda/octavia/1...,NaN,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1603226288,RUB,1101037972,"{""id"":""4561004"",""displacement"":1798,""engine_ty...",EUROPEAN


Для формирования общего датасета, согласно сравнения, необходимо сделать следующие модификация датасетов:

#### Переименовать в df:
- Комплектация - complectation_dict

#### Убрать из df:
- hidden
- start_date

#### Добавить в df: 
- VENDOR
- train = 1

#### Убрать из test:
- parsing_unixtime
- car_url
- priceCurrency
- image
- sell_id
- super_gen

#### Переименовать в test:
- models_name - model
#### Добавить в test:
- train = 0

#### Price - целевая переменная 

### Чистим датасеты от лишних столбцов

In [11]:
df = df.rename({'Комплектация':'complectation_dict'}, axis='columns')

In [12]:
df = df.drop(['hidden', 'start_date'], axis = 1)

Разметим датасеты, добавив столбец train

In [13]:
df['train'] = 1
test['train'] = 0

In [14]:
test = test.rename({'model_name':'model'}, axis='columns')

In [15]:
test = test.drop(['parsing_unixtime', 'car_url', 'priceCurrency', 'image', 
                  'sell_id', 'super_gen','model_info','equipment_dict'], axis = 1)

In [16]:
df_col = df.columns
test_col = test.columns

In [17]:
common_col = []
t_col = []
count = 0
t = 0
for col in test_col:
    if col in df_col:
        common_col.append(col)
        count +=1
    else:
        t_col.append(col)
        t +=1
print(count, common_col)
print(t, t_col)

24 ['bodyType', 'brand', 'color', 'complectation_dict', 'description', 'engineDisplacement', 'enginePower', 'fuelType', 'mileage', 'modelDate', 'model', 'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня', 'train']
1 ['vendor']


Остался один столбец vendor, который отсутвует в тренировочном датасете. Добавим его

In [18]:
vendors = {'BMW':'EUROPEAN',
 'VOLKSWAGEN':'EUROPEAN',
 'MERCEDES': 'EUROPEAN',
 'HYUNDAI':'KOREA',
 'NISSAN':'JAPANESE',
 'KIA':'KOREA',
 'AUDI':'EUROPEAN',
 'FORD':'USA',
 'RENAULT':'EUROPEAN',
 'MITSUBISHI':'JAPANESE',
 'TOYOTA':'JAPANESE',
 'CHEVROLET':'USA',
 'SKODA':'EUROPEAN',
 'OPEL':'EUROPEAN',
 'MAZDA':'JAPANESE',
 'VOLVO':'EUROPEAN',
 'PEUGEOT':'EUROPEAN',
 'LEXUS':'JAPANESE',
 'CITROEN':'EUROPEAN',
 'SUZUKI':'JAPANESE',
 'HONDA':'JAPANESE',
 'SUBARU':'JAPANESE',
 'INFINITI':'JAPANESE',
 'DAEWOO':'KOREA',
 'PORSCHE':'EUROPEAN',
 'JEEP':'USA',
 'CHERY':'CHINA',
 'MINI':'EUROPEAN',
 'CADILLAC':'USA',
 'DODGE':'USA',
 'JAGUAR':'EUROPEAN',
 'CHRYSLER':'USA',
 'GEELY':'CHINA'}

df['vendor'] = df['brand']
for brand in df.brand:
    df['vendor'] = df['vendor'].replace(brand, vendors[brand])

In [19]:
# test['price'] = 0

Объединим датасеты

В тренировочном датасете 227 пропусков в цене. Уберем их

In [20]:
df['price'] = df['price'].fillna(0)
df = df[df['price'] != 0]
df['price'].isna().sum()

0

In [21]:
data = test.append(df, sort=False).reset_index(drop=True)

In [22]:
df['price'].isna().sum()

0

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96317 entries, 0 to 96316
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              96315 non-null  object 
 1   brand                 96317 non-null  object 
 2   color                 96317 non-null  object 
 3   complectation_dict    68049 non-null  object 
 4   description           94463 non-null  object 
 5   engineDisplacement    96315 non-null  object 
 6   enginePower           96315 non-null  object 
 7   fuelType              96317 non-null  object 
 8   mileage               96317 non-null  int64  
 9   modelDate             96315 non-null  float64
 10  model                 96317 non-null  object 
 11  name                  96315 non-null  object 
 12  numberOfDoors         96315 non-null  float64
 13  productionDate        96317 non-null  int64  
 14  vehicleConfiguration  96315 non-null  object 
 15  vehicleTransmission

Всего получилось 96316 строк в общем датасете. Большая часть данных типа Object, что в большинстве своем носит, скорее всего, категориальный характер данных.



### 2. EDA & Feature engineering

Проверким столбцы на пропуски

In [24]:
def nan_check(col):
    return data[col].isna().sum()

column = data.columns
for i in column:
    print(i, nan_check(i))

bodyType 2
brand 0
color 0
complectation_dict 28268
description 1854
engineDisplacement 2
enginePower 2
fuelType 0
mileage 0
modelDate 2
model 0
name 2
numberOfDoors 2
productionDate 0
vehicleConfiguration 2
vehicleTransmission 2
vendor 0
Владельцы 11406
Владение 68358
ПТС 334
Привод 2
Руль 0
Состояние 61631
Таможня 0
train 0
price 34686


In [25]:
def fillna(col):
    data[col] = data[col].fillna('None')


Отметим пропуски меткой None

In [26]:
column = data.columns
for i in column:
    fillna(i)

Для столбцов, в которых по 2 пропуска, будем удалять целиком строки.

In [27]:
df = data[(data['bodyType']!='None')&(data['engineDisplacement']!='None')&
     (data['enginePower']!='None')&(data['modelDate']!='None')&
     (data['name']!='None')&(data['numberOfDoors']!='None')&
     (data['vehicleConfiguration']!='None')&(data['vehicleTransmission']!='None')&
    (data['Привод']!='None')]

Добавим столбец с рейтингм продажи автомобилей за октябрь 2020 года

In [28]:
brand_rating = {'BMW':'0',
 'VOLKSWAGEN':'6',
 'MERCEDES':'0',
 'HYUNDAI':'3',
 'NISSAN':'8',
 'KIA':'2',
 'AUDI':'18',
 'FORD':'48',
 'RENAULT':'4',
 'MITSUBISHI':'11',
 'TOYOTA':'7',
 'CHEVROLET':'41',
 'SKODA':'5',
 'OPEL':'37',
 'MAZDA':'12',
 'VOLVO':'19',
 'PEUGEOT':'27',
 'LEXUS':'14',
 'CITROEN':'29',
 'SUZUKI':'21',
 'HONDA':'35',
 'SUBARU':'24',
 'INFINITI':'30',
 'DAEWOO':'0',
 'PORSCHE':'26',
 'JEEP':'34',
 'CHERY':'17',
 'MINI':'0',
 'CADILLAC':'33',
 'DODGE':'0',
 'JAGUAR':'38',
 'CHRYSLER':'47',
 'GEELY':'13'}

df['brand_rating'] = df['brand']
for brand in data.brand:
    df['brand_rating'] = df['brand_rating'].replace(brand, brand_rating[brand])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Сохраним подгтовленный датасет в файл, чтоб не тратить время на обработку

In [29]:
df.to_csv(PATH_to_file+'for_EDA.csv', index=False)

### 2.1 Проверим каждый столбец отдельно

In [5]:
import pandas as pd
from datetime import datetime, date, time

PATH_to_file = './data/'
RANDOM_SEED = 42
now = datetime.now()

In [6]:
df = pd.read_csv(PATH_to_file+'for_EDA.csv')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (6,23,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### BodyType

Посмотрим форму представления признака

In [ ]:
df['bodyType'].unique()

Представим в форме наименования кузова

In [8]:
df['bodyType'] = df['bodyType'].apply(lambda x: (x.split(' ')[0]).lower())

In [9]:
df['bodyType']

0            лифтбек
1            лифтбек
2            лифтбек
3            лифтбек
4            лифтбек
            ...     
96310    внедорожник
96311    внедорожник
96312    внедорожник
96313    внедорожник
96314    внедорожник
Name: bodyType, Length: 96315, dtype: object

#### Brand

В столбце brand никаких сюрпризов

In [10]:
df['brand'].unique()

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI',
       'CADILLAC', 'CHERY', 'CHEVROLET', 'CHRYSLER', 'CITROEN', 'DAEWOO',
       'DODGE', 'FORD', 'GEELY', 'HYUNDAI', 'JAGUAR', 'JEEP', 'KIA',
       'MAZDA', 'MINI', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SUBARU',
       'SUZUKI'], dtype=object)

#### Описание (description)

In [12]:
df['description'][1]

'ЛОТ: 01217195\nАвтопрага Север\nДанный автомобиль прошел диагностику по 147 пунктам и имеет сертификат технической гарантии.\n\n\nВы можете получить скидку на данный автомобиль до 50000 рублей. Подробности уточняйте у менеджеров отдела продаж.\n\nГК FAVORIT MOTORS строит свою работу на принципах открытости и честности! Нам доверяют более 25 лет.\n\n- Автомобиль продается официальным дилером FAVORIT MOTORS.\n- Покупка и обслуживание у официального дилера.\n- Проведена предпродажная подготовка,\n- Гарантия юридической чистоты.\n- Круглосуточная техническая поддержка с бесплатным эвакуатором.\n\n- Автомобиль можно приобрести за наличный расчет, безналичным платежом, а также мы примем вашу машину по рыночной стоимости в рамках программы трейд-ин и при необходимости — выплатим за нее кредит.\n\n- Возможна покупка в кредит: специальные предложения по кредитованию и страхованию.\n- Оформление кредита по двум документам.\n- КАСКО не обязательно.\n\nПриглашаем на бесплатный тест-драйв!\n\nГруп

Достаточно длинное описание. Нет типовой формы, поэтому каждое описание разное. Что либо полезное выделить тяжело. В каждом упоминается trad-in и возможность оформления кредита.

Возьмем за признак длину описания

In [17]:
df['description'] = df['description'].apply(lambda x: len(x))

#### Двигатель (engineDisplacement)

In [ ]:
df['engineDisplacement'].unique()

Наименования двигателей вряд ли имеют решающее значение. Рассмотренный столбец удалим

In [20]:
df = df.drop('engineDisplacement', axis = 1)

#### Мощность (enginePower)

Мощность двигателя, как правило играет важную роль. Забегая вперед скажу, что мощность мы возьмём из столбца Name

In [ ]:
df['enginePower'].unique()

#### Тип топлива (fuelType)

In [25]:
df['fuelType'].unique()

array(['бензин', 'дизель', 'гибрид', 'электро', 'газ'], dtype=object)

Здесь всё хорошо, в чистом виде категориальный признак

#### Пробег (mileage)

In [26]:
df['mileage'].unique()

array([ 74000,  60563,  88000, ..., 179067,  81782, 194620], dtype=int64)

Числовой столбец со значениями пробега

#### Год выпуска модели (modelDate )

In [27]:
df['modelDate'].unique()

array([2013., 2017., 2008., 2009., 2016., 2012., 2015., 2010., 2006.,
       2000., 2007., 1994., 2004., 1999., 2005., 1976., 2001., 1969.,
       1996., 1998., 1989., 1934., 2014., 2011., 2018., 1986., 1997.,
       1990., 2019., 2002., 1991., 1987., 1980., 1982., 1938., 1988.,
       2003., 1983., 1978., 1979., 1984., 1992., 1995., 1993., 1985.,
       1974., 1966., 1977., 1981., 1972., 1968., 1975., 1949., 1937.,
       1936., 1973., 1959., 1958., 2020., 1965., 1971., 1904., 1963.,
       1955., 1951., 1960., 1967., 1970., 1957., 1942., 1927., 1908.,
       1932., 1948., 1950.])

In [31]:
df['modelDate'] = df['modelDate'].apply(lambda x: int(x))

Столбец с датами начала выпуска модели. Преобразуем в целочисленные и добавим столбец давности начала производства в месяцах

In [33]:
df['modelDate'] = pd.to_datetime(df['modelDate'], format = '%Y')
df['model_age'] = (now - df['modelDate'])

In [35]:
df['model_age'] = df['model_age'].apply(lambda x: int(int(str(x).split(' ')[0])/30))

Столбец modelDate больше не нужен - удаляем

In [38]:
df = df.drop('modelDate', axis = 1)

#### Модель (model)

In [41]:
df['model'].unique()

array(['OCTAVIA', 'SUPERB', 'OCTAVIA_RS', ..., 'TANK', 'KARMANN_GHIA',
       'TYPE_2'], dtype=object)

#### Имя (name)

Посмотрим представление данных

In [42]:
df['name'][1]

'1.6 MT (110 л.с.)'

In [43]:
df['name'] = df['name'].apply(lambda x: x.split('(')[1])
df['name'] = df['name'].apply(lambda x: x.split(')'))

Добавим столбец мощности

In [48]:
df['HP'] = df['name'].apply(lambda x: x[0].split(' ')[0])
df['HP'] = df['HP'].apply(lambda x: int(x))

#### Количество дверей (numberOfDoors)

In [52]:
df['numberOfDoors'].unique()

array([5., 4., 2., 3., 0.])

Оставим, как катeгориальный признак

#### Дата выпуска (productionDate)

In [53]:
df['productionDate'].unique()

array([2014, 2017, 2012, 2011, 2019, 2018, 2010, 2020, 2016, 2013, 2006,
       2007, 2015, 2005, 2008, 2009, 1997, 2004, 2002, 1987, 2003, 2001,
       1976, 2000, 1998, 1995, 1999, 1993, 1939, 1996, 1984, 1990, 1991,
       1992, 1989, 1982, 1985, 1994, 1938, 1981, 1988, 1983, 1980, 1986,
       1978, 1970, 1979, 1977, 1972, 1975, 1969, 1950, 1953, 1949, 1937,
       1959, 1968, 1936, 1904, 1974, 1967, 1961, 1960, 1965, 1963, 1957,
       1952, 1973, 1948, 1955, 1964, 1951, 1943, 1927, 1928, 1923, 1932,
       1956, 1971], dtype=int64)

#### Конфигурация (vehicleConfiguration)

In [55]:
df['vehicleConfiguration'].nunique()

1699

In [ ]:
df[df['vehicleConfiguration'] == 'None']

Во всём датасете 1699 различных конфигураций автомобилей. Оставим, как категория

#### Трансмиссия (vehicleTransmission)

In [59]:
df['vehicleTransmission'].unique()

array(['ROBOT', 'MECHANICAL', 'AUTOMATIC', 'VARIATOR'], dtype=object)

In [58]:
df['vehicleTransmission'] = df['vehicleTransmission'].replace('роботизированная','ROBOT')
df['vehicleTransmission'] = df['vehicleTransmission'].replace('механическая','MECHANICAL')
df['vehicleTransmission'] = df['vehicleTransmission'].replace('автоматическая','AUTOMATIC')
df['vehicleTransmission'] = df['vehicleTransmission'].replace('вариатор','VARIATOR')

Категориальный признак

#### Владельцы

In [61]:
df['Владельцы'].value_counts()

3.0            21898
3 или более    15957
1.0            14873
2.0            13452
None           11406
1 владелец      9459
2 владельца     9270
Name: Владельцы, dtype: int64

У 11406 отсутсвует упоминание о владельцах

In [64]:
df[df['Владельцы'] == 'None']['productionDate'].value_counts()

2020    11191
2019      208
2018        7
Name: productionDate, dtype: int64

Видим, что 11191 является 2020 года выпуска, таким образом является новой. Можно добавить иникатор новой машины

In [67]:
df['owners'] = df['Владельцы'].apply(lambda x: (str(x))[0])
df['owners'] = df['owners'].replace('N',0)
df['owners'] = df['owners'].apply(lambda x: int(x))

In [68]:
df['owners'].unique()

array([3, 1, 2, 0], dtype=int64)

Удалим ставший ненужным столбец Владельцы

In [69]:
df = df.drop('Владельцы', axis = 1)

#### Владение